In [1]:
import os
import sounddevice as sd
import whisper
from scipy.io.wavfile import write


In [8]:
def record():
    freq = 44100
    duration = 60
    print("Recording...")
    recording = sd.rec(int(duration * freq), samplerate=freq, channels=1)
    sd.wait()
    write("recording.wav", freq, recording)
    return "recording.wav"

# def transcribe():
#     model_w = whisper.load_model("small")
#     result = model_w.transcribe("recording.wav")
#     trans_text = result["text"]
#     print("Transcription:", trans_text)
#     return trans_text

In [4]:
def transcribe():
    model_w = whisper.load_model("large",device="cuda")
    result = model_w.transcribe("recording.wav", task="translate")
    trans_text = result["text"]
    print("Transcription:", trans_text)
    return trans_text

In [5]:
# record()
transcribe()

Transcription:  My Lord, Ram has a desire to learn and study a lot. But he sleeps only when he opens a book. At this time, Ram is giving a gift to God. Instead of writing a book to God and reading it, God asks him, can you come and talk to me? Then God tells him about podcast. Since then, Ram sleeps only when he opens a book. He sleeps only when he listens to podcasts. If you are like Ram, save this video and start the podcast I am telling. This is the idea of CEO. He would have asked questions that people who are not able to think of would not ask. This is WTF. He brings top leaders of each industry and brings them all the information in 2 hours. Both are English.


' My Lord, Ram has a desire to learn and study a lot. But he sleeps only when he opens a book. At this time, Ram is giving a gift to God. Instead of writing a book to God and reading it, God asks him, can you come and talk to me? Then God tells him about podcast. Since then, Ram sleeps only when he opens a book. He sleeps only when he listens to podcasts. If you are like Ram, save this video and start the podcast I am telling. This is the idea of CEO. He would have asked questions that people who are not able to think of would not ask. This is WTF. He brings top leaders of each industry and brings them all the information in 2 hours. Both are English.'

In [6]:
def transcribes():
    model_w = whisper.load_model("medium",device="cuda")
    result = model_w.transcribe("recording.wav", task="translate")
    trans_text = result["text"]
    print("Transcription:", trans_text)
    return trans_text

In [8]:
import torch
print(torch.cuda.is_available())


True


In [10]:
transcribes()

Transcription:  I want to learn a lot from Ram. But he will fall asleep when I open a book. At this time, God gives a gift to Ram. Instead of writing a book to God, he asks me if I can talk to him. God says that he wants a podcast. Since then, Ram has been sleeping listening to podcasts. If you are interested, start the podcast I am talking about. I have a question for people who have not heard the podcast. WTF! He brings the top leaders of every industry and brings the best information in 2 hours. Why is everything in English?


' I want to learn a lot from Ram. But he will fall asleep when I open a book. At this time, God gives a gift to Ram. Instead of writing a book to God, he asks me if I can talk to him. God says that he wants a podcast. Since then, Ram has been sleeping listening to podcasts. If you are interested, start the podcast I am talking about. I have a question for people who have not heard the podcast. WTF! He brings the top leaders of every industry and brings the best information in 2 hours. Why is everything in English?'

In [1]:
from langchain import hub

prompt = hub.pull("hwchase17/openai-functions-agent")
print(prompt)


d:\projects\generative ai\ai meeting\myenv\Lib\site-packages\langsmith\client.py:288: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


input_variables=['agent_scratchpad', 'input'] optional_variables=['chat_history'] input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')]

In [6]:
print(prompt.pretty_repr())


================================ System Message ================================

You are a helpful assistant

============================= Messages Placeholder =============================

{chat_history}

================================ Human Message =================================

{input}

============================= Messages Placeholder =============================

{agent_scratchpad}


In [1]:
import sounddevice as sd
import soundfile as sf
import numpy as np
import threading
import time

DURATION = 10.0  # seconds
SR = 48000       # target sample rate (change if your devices require different)
CHANNELS_SYS = 2 # usually stereo for system audio
CHANNELS_MIC = 1 # typical mic mono (could be 2)

def find_wasapi_loopback():
    devs = sd.query_devices()
    wasapi = sd.query_hostapis()
    # Try to find a device with 'loopback' in its name and hostapi = WASAPI
    for i, d in enumerate(devs):
        name = d['name'].lower()
        if 'loopback' in name or 'loop back' in name:
            return i, d
    # fallback: find devices under WASAPI hostapi and pick one that looks like speakers
    for i, d in enumerate(devs):
        if d['hostapi'] == sd.default.hostapi:
            n = d['name'].lower()
            if 'speaker' in n or 'output' in n or 'realtek' in n or 'headphone' in n:
                # Many WASAPI devices have a separate loopback device index — you might need to inspect list manually.
                # If you don't find explicit loopback, try enabling 'Stereo Mix' or using a virtual cable.
                pass
    return None, None

def choose_devices():
    print("Available devices (index: name):")
    for i, d in enumerate(sd.query_devices()):
        print(i, ":", d['name'], "IN channels:", d['max_input_channels'], "OUT channels:", d['max_output_channels'])
    # Try auto-find
    loop_idx, loop_dev = find_wasapi_loopback()
    mic_idx = None
    # choose first device with input channels > 0 (not loopback) for mic
    for i, d in enumerate(sd.query_devices()):
        if d['max_input_channels'] > 0 and i != loop_idx:
            mic_idx = i
            break
    return loop_idx, mic_idx

def record_stream(device_idx, channels, samplerate, duration, buffer_list, name):
    def callback(indata, frames, time_info, status):
        if status:
            print(f"{name} status:", status)
        buffer_list.append(indata.copy())

    with sd.InputStream(device=device_idx, channels=channels, samplerate=samplerate,
                        blocksize=1024, callback=callback):
        sd.sleep(int(duration * 1000))

def flatten_buffers(buffers):
    if not buffers:
        return np.empty((0,0), dtype=np.float32)
    return np.vstack(buffers)

def main():
    loop_idx, mic_idx = choose_devices()
    if loop_idx is None:
        print("Could not auto-find a loopback device. Look for devices with 'loopback' or enable Stereo Mix / use a virtual cable.")
        return
    print("Using loopback device index:", loop_idx, "and mic index:", mic_idx)

    sys_buf = []
    mic_buf = []

    t1 = threading.Thread(target=record_stream, args=(loop_idx, CHANNELS_SYS, SR, DURATION, sys_buf, "SYSTEM"))
    t2 = threading.Thread(target=record_stream, args=(mic_idx, CHANNELS_MIC, SR, DURATION, mic_buf, "MIC"))

    print("Starting recording for", DURATION, "seconds...")
    t1.start(); t2.start()
    t1.join(); t2.join()
    print("Recording done.")

    sys_audio = flatten_buffers(sys_buf)   # shape (n_frames, CHANNELS_SYS)
    mic_audio = flatten_buffers(mic_buf)   # shape (n_frames, CHANNELS_MIC)

    # Align lengths
    n = min(len(sys_audio), len(mic_audio))
    if n == 0:
        print("No data captured from one of the devices.")
        return
    sys_audio = sys_audio[:n]
    mic_audio = mic_audio[:n]

    # If mic is mono and system is stereo, expand mic to 2 channels or make stereo pair
    if mic_audio.ndim == 1:
        mic_audio = mic_audio.reshape(-1, 1)
    if mic_audio.shape[1] == 1 and sys_audio.shape[1] == 2:
        mic_stereo = np.repeat(mic_audio, 2, axis=1)
    else:
        mic_stereo = mic_audio

    # Save a stereo file where left=system, right=mic (simple mapping)
    out_stereo = np.zeros((n, 2), dtype=np.float32)
    # map system to left channel (mix both system channels if stereo)
    out_stereo[:, 0] = sys_audio.mean(axis=1) if sys_audio.shape[1] > 1 else sys_audio[:,0]
    # map mic to right channel
    out_stereo[:, 1] = mic_stereo[:, 0]

    sf.write('system_left_mic_right.wav', out_stereo, SR)
    print("Wrote system_left_mic_right.wav")

    # Also write a simple mixed mono file (system + mic)
    mono_mix = out_stereo.mean(axis=1)
    # normalize to avoid clipping
    maxv = np.max(np.abs(mono_mix))
    if maxv > 0.99:
        mono_mix = mono_mix / (maxv + 1e-9)
    sf.write('mixed_mono.wav', mono_mix, SR)
    print("Wrote mixed_mono.wav")

if __name__ == "__main__":
    main()


Available devices (index: name):
0 : Microsoft Sound Mapper - Input IN channels: 2 OUT channels: 0
1 : Microphone Array (Realtek(R) Au IN channels: 2 OUT channels: 0
2 : Stereo Mix (Realtek(R) Audio) IN channels: 2 OUT channels: 0
3 : Microsoft Sound Mapper - Output IN channels: 0 OUT channels: 2
4 : Speakers (Realtek(R) Audio) IN channels: 0 OUT channels: 2
5 : Primary Sound Capture Driver IN channels: 2 OUT channels: 0
6 : Microphone Array (Realtek(R) Audio) IN channels: 2 OUT channels: 0
7 : Stereo Mix (Realtek(R) Audio) IN channels: 2 OUT channels: 0
8 : Primary Sound Driver IN channels: 0 OUT channels: 2
9 : Speakers (Realtek(R) Audio) IN channels: 0 OUT channels: 2
10 : Speakers (Realtek(R) Audio) IN channels: 0 OUT channels: 2
11 : Microphone Array (Realtek(R) Audio) IN channels: 2 OUT channels: 0
12 : Stereo Mix (Realtek(R) Audio) IN channels: 2 OUT channels: 0
13 : Microphone (Realtek HD Audio Mic input) IN channels: 2 OUT channels: 0
14 : Speakers 1 (Realtek HD Audio output w

In [2]:
import sounddevice as sd
import soundfile as sf
import numpy as np
import threading

# --- Choose your devices here ---
SYSTEM_AUDIO_DEVICE = 2   # Stereo Mix (Realtek)
MIC_AUDIO_DEVICE = 13     # Microphone (Realtek HD Audio Mic input)
SR = 44100                # Common sample rate
DURATION = 10             # seconds
# --------------------------------

def record_audio(device_idx, channels, samplerate, duration, buffer_list, label):
    def callback(indata, frames, time_info, status):
        if status:
            print(f"{label} status:", status)
        buffer_list.append(indata.copy())

    with sd.InputStream(device=device_idx, channels=channels, samplerate=samplerate, callback=callback):
        sd.sleep(int(duration * 1000))

def flatten(buffers):
    return np.vstack(buffers) if buffers else np.empty((0,))

def main():
    sys_buf = []
    mic_buf = []

    sys_thread = threading.Thread(target=record_audio, args=(SYSTEM_AUDIO_DEVICE, 2, SR, DURATION, sys_buf, "SYSTEM"))
    mic_thread = threading.Thread(target=record_audio, args=(MIC_AUDIO_DEVICE, 2, SR, DURATION, mic_buf, "MIC"))

    print(f"Recording system (device {SYSTEM_AUDIO_DEVICE}) + mic (device {MIC_AUDIO_DEVICE}) for {DURATION}s...")
    sys_thread.start(); mic_thread.start()
    sys_thread.join(); mic_thread.join()
    print("Done recording!")

    sys_audio = flatten(sys_buf)
    mic_audio = flatten(mic_buf)

    n = min(len(sys_audio), len(mic_audio))
    sys_audio, mic_audio = sys_audio[:n], mic_audio[:n]

    # Mix both (average) into a mono file
    mix = (sys_audio.mean(axis=1) + mic_audio.mean(axis=1)) / 2
    mix = mix / np.max(np.abs(mix))  # normalize

    sf.write("system_mix.wav", sys_audio, SR)
    sf.write("mic.wav", mic_audio, SR)
    sf.write("mixed.wav", mix, SR)

    print("✅ Saved: system_mix.wav, mic.wav, mixed.wav")

if __name__ == "__main__":
    main()


Exception in thread Thread-6 (record_audio):
Traceback (most recent call last):
  File "c:\Users\SELVAGANAPATHI\anaconda3\Lib\threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "c:\Users\SELVAGANAPATHI\anaconda3\Lib\threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\SELVAGANAPATHI\AppData\Local\Temp\ipykernel_24212\1579394000.py", line 19, in record_audio
  File "c:\Users\SELVAGANAPATHI\anaconda3\Lib\site-packages\sounddevice.py", line 1440, in __init__
    _StreamBase.__init__(self, kind='input', wrap_callback='array',
  File "c:\Users\SELVAGANAPATHI\anaconda3\Lib\site-packages\sounddevice.py", line 909, in __init__
    _check(_lib.Pa_OpenStream(self._ptr, iparameters, oparameters,
  File "c:\Users\SELVAGANAPATHI\anaconda3\Lib\site-packages\sounddevice.py", line 2804, in _check
    raise PortAudioError(errormsg, err)
sounddevice.PortAudioError: Error opening InputStream: Invalid device [PaErrorCode -9996]


Recording system (device 2) + mic (device 13) for 10s...
Done recording!


AxisError: axis 1 is out of bounds for array of dimension 1

In [3]:
import sounddevice as sd
for i, d in enumerate(sd.query_devices()):
    print(i, ":", d['name'], "|", d['hostapi'], "| IN:", d['max_input_channels'], "OUT:", d['max_output_channels'])



0 : Microsoft Sound Mapper - Input | 0 | IN: 2 OUT: 0
1 : Microphone Array (Realtek(R) Au | 0 | IN: 2 OUT: 0
2 : Stereo Mix (Realtek(R) Audio) | 0 | IN: 2 OUT: 0
3 : Microsoft Sound Mapper - Output | 0 | IN: 0 OUT: 2
4 : Speakers (Realtek(R) Audio) | 0 | IN: 0 OUT: 2
5 : Primary Sound Capture Driver | 1 | IN: 2 OUT: 0
6 : Microphone Array (Realtek(R) Audio) | 1 | IN: 2 OUT: 0
7 : Stereo Mix (Realtek(R) Audio) | 1 | IN: 2 OUT: 0
8 : Primary Sound Driver | 1 | IN: 0 OUT: 2
9 : Speakers (Realtek(R) Audio) | 1 | IN: 0 OUT: 2
10 : Speakers (Realtek(R) Audio) | 2 | IN: 0 OUT: 2
11 : Microphone Array (Realtek(R) Audio) | 2 | IN: 2 OUT: 0
12 : Stereo Mix (Realtek(R) Audio) | 2 | IN: 2 OUT: 0
13 : Microphone (Realtek HD Audio Mic input) | 3 | IN: 2 OUT: 0
14 : Speakers 1 (Realtek HD Audio output with HAP) | 3 | IN: 0 OUT: 2
15 : Speakers 2 (Realtek HD Audio output with HAP) | 3 | IN: 0 OUT: 2
16 : PC Speaker (Realtek HD Audio output with HAP) | 3 | IN: 2 OUT: 0
17 : Microphone Array (Realtek HD

In [4]:
#ithu ok
import sounddevice as sd
import soundfile as sf
import numpy as np

device_name = None
for i, d in enumerate(sd.query_devices()):
    if "stereo mix" in d['name'].lower():
        device_name = i
        print("Found Stereo Mix at index", i, ":", d['name'])
        break

if device_name is None:
    raise RuntimeError("Stereo Mix device not found")

# Then use it in the recording
SR = 44100
DURATION = 10

print("Recording from Stereo Mix...")
data = sd.rec(int(DURATION*SR), samplerate=SR, channels=2, device=DEVICE_IDX)
sd.wait()

# Normalize to avoid clipping
data = data / np.max(np.abs(data))

sf.write("system_plus_mic.wav", data, SR)
print("Saved system_plus_mic.wav")


Found Stereo Mix at index 1 : Stereo Mix (Realtek(R) Audio)
Recording from Stereo Mix...
Saved system_plus_mic.wav


In [1]:
import sounddevice as sd

# Look for a device that contains "Stereo Mix" in the name
device_name = None
for i, d in enumerate(sd.query_devices()):
    if "stereo mix" in d['name'].lower():
        device_name = i
        print("Found Stereo Mix at index", i, ":", d['name'])
        break

if device_name is None:
    raise RuntimeError("Stereo Mix device not found")

# Then use it in the recording
SR = 44100
DURATION = 10
data = sd.rec(int(DURATION*SR), samplerate=SR, channels=2, device=device_name)
sd.wait()


Found Stereo Mix at index 1 : Stereo Mix (Realtek(R) Audio)


In [1]:
for i, d in enumerate(sd.query_devices()):
    if "stereo mix" in d["name"].lower():
        device_idx = i
        print(f"✅ Found Stereo Mix at index {i}: {d['name']}")
        break

NameError: name 'sd' is not defined